# TODO
- Check if the series needs / benefits from a BoxCox transform

In [ ]:
library(forecast)

In [ ]:
loadData <- function(dataFolder) {
    files <- list.files(dataFolder)
    data <- list()
    for(file in files) {    
        df <- read.csv(paste0(dataFolder, "/", file), stringsAsFactors=F)    
        minYear <- min(df$Year)
        complaintType <- substr(file,1,(nchar(file))-4)    
        tsObject <- ts(df$Complaints, start=c(minYear, 1), frequency = 12)
        data[[complaintType]] <- tsObject
    }
    data
}
data <- loadData("../../data/topNComplaints")

In [ ]:
series <- data[["Non Burning of Street Lights"]]

In [ ]:
tsdisplay(series)

In [ ]:
# data before 2012 are too few to consider
series <- window(series, start=c(2012, 1), end=c(2016, 6))
tsdisplay(series)

## Cleaning up data 

Although this data looks like it doesn't have any outliers, let's take a look at where the potential extreme values are

In [ ]:
plot(series, col="red", lty=2)
lines(tsclean(series), lty=1)
legend("topright", col=c("red", "black"), lty=c(2,1), legend=c("Original", "Cleaned"))

Taking a call here that the data doesn't contain any outliers, so we're leaving the data as it is

## Decomposition

In [ ]:
# first try a static seasonal component
plot(stl(series, s.window="periodic"))

The trend component is the most significant here, so the series probably needs some differencing. Strangely, there is also a seasonal component. Let's take varying s.window to see if changes over time. 

In [ ]:
old.par <- par(mfrow=c(2, 2), mar=c(3,3,3,3))
plot(stl(series, s.window=3)$time.series[, 1], main="Seasonal Component with s.window = 3")
plot(stl(series, s.window=6)$time.series[, 1], main="Seasonal Component with s.window = 6")
plot(stl(series, s.window=10)$time.series[, 1], main="Seasonal Component with s.window = 10")
plot(stl(series, s.window=12)$time.series[, 1], main="Seasonal Component with s.window = 12")
par(old.par)

Looks like the seasonal component is there, but $s.window=3$ suggests that it is not as significant

In [ ]:
seasonal <- stl(series, s.window="periodic")$time.series[, 1] # change s.window
plot(seasonal, col="grey")
month <- 11 # change this to month you want
for(i in 2012:2016) {    
    abline(v=(month-1)/12 + i, lty=2)
}

**Looks like it peaks in November. **

Let us then do a seasonal adjustment of the data. All further analysis should be done on this data

In [ ]:
stl.fit <- stl(series, s.window="periodic")
series.adj <- seasadj(stl.fit)
tsdisplay(series.adj)

## Forecasting
### ARIMA models - estimating p, d, q

First, let us estimate $d$. This is done by looking at the ACF of the data.

In [ ]:
Acf(series.adj)

In [ ]:
# the above series is a classic example of a series that requires a diff of order 1, 
# so let's try that out and take a look at the Acf to see if it is overdifferenced
series.diff <- diff(series.adj, lag=1, differences = 1)
tsdisplay(series.diff)

In [ ]:
# the series looks good!
# let's take a look at the standard deviation as well
sd(series.adj)

In [ ]:
sd(series.diff)

In [ ]:
# looks good - it has decreased. Since stationary series return to the mean, let's take a look at that as well
plot(series.diff, col="grey")
# a 2x4 MA
lines(ma(ma(series.diff, order=2), order=4))
abline(mean(series.diff), 0, col="blue", lty=2)

In [ ]:
# let's verify once wheather d=1
ndiffs(series.adj)

Next, we need to estimate p and q. To do this, we take a look at the PACF of the data. Note that this analysis is done on the differenced data. If we decide to fit a model with d=0, then we need to perform this analysis for the un-differenced data as well

In [ ]:
# for d=0
Pacf(series.adj)

In [ ]:
# looks like a AR(1) and a MA(5) process
# take a look at the d=1
Pacf(series.diff)

In [ ]:
# this looks like a MA(11) and a AR(4) process

#### Building candidate models

In [ ]:
modelArima <- function(series, order, h, testData = NULL) {
    fit <- Arima(series, order=order)
    print(summary(fit))
    predictions <- forecast(fit, h)
    plot(predictions)
    if(!is.null(testData)) {
        lines(testData, col="red", lty=2)
        print(accuracy(predictions, testData))
    }
    # check if residuals looklike white noise
    Acf(residuals(fit), main="Residuals")
    # portmantaeu test
    print(Box.test(residuals(fit), lag=24, fitdf=4, type="Ljung"))
}

In [ ]:
# split the series into a test and a train set
series.train <- window(series.adj, end=c(2015, 6))
series.test <- window(series.adj, start=c(2015, 7))

In [ ]:
# with d=0, order=(1, 0, 5)
modelArima(series.train, c(1, 0, 5), length(series.test), series.test)

In [ ]:
# with d=1, order=(4, 1, 11)
modelArima(series.train, c(4, 1, 11), length(series.test), series.test)

In [ ]:
# fiddle with p and q, with d=1
modelArima(series.train, c(5, 1, 11), length(series.test), series.test)

In [ ]:
modelArima(series.train, c(4, 1, 12), length(series.test), series.test)

In [ ]:
modelArima(series.train, c(4, 1, 10), length(series.test), series.test)

In [ ]:
modelArima(series.train, c(3, 1, 11), length(series.test), series.test)

In [ ]:
# MRK, take it from here. 
# quick info: 
# series.adj is seasonally adjusted data
# series.train is the training sample of series.adj
# series.test is the test sample of series.adj